# PyKale Tutorial: Drug-Target Interaction: DeepDTA

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/pykale/pykale/HEAD?filepath=examples%2Fbindingdb_deepdta%2Ftutorial.ipynb) 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pykale/pykale/blob/main/examples/bindingdb_deepdta/tutorial.ipynb)


## Setup

The first few blocks of code are necessary to set up the notebook execution environment and import the required modules, including PyKale.

This checks if the notebook is running on Google Colab and installs required packages.

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip install -q condacolab
    import condacolab
    condacolab.install()    

    !git clone -b bindingdb_deepdta_tutorial https://github.com/pykale/pykale.git

    !conda env update -n base -f pykale/environment.yml
    
    %cd  pykale/examples/bindingdb_deepdta
else:
    print('Not running on CoLab')


In [ ]:
import pytorch_lightning as pl
import torch
from config import get_cfg_defaults
from model import get_model
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger
from torch.utils.data import DataLoader

from kale.loaddata.tdc_datasets import BindingDBDataset

# Set configs, logger and device

In [ ]:
cfg = get_cfg_defaults()
cfg.merge_from_file(args.cfg)
cfg.freeze()
tb_logger = TensorBoardLogger("tb_logs", name=cfg.DATASET.NAME)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set dataset

In [ ]:
train_dataset = BindingDBDataset(name=cfg.DATASET.NAME, split="train", path=cfg.DATASET.PATH)
val_dataset = BindingDBDataset(name=cfg.DATASET.NAME, split="valid", path=cfg.DATASET.PATH)
test_dataset = BindingDBDataset(name=cfg.DATASET.NAME, split="test", path=cfg.DATASET.PATH)
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=cfg.SOLVER.TRAIN_BATCH_SIZE)
val_loader = DataLoader(dataset=val_dataset, shuffle=True, batch_size=cfg.SOLVER.TEST_BATCH_SIZE)
test_loader = DataLoader(dataset=test_dataset, shuffle=True, batch_size=cfg.SOLVER.TEST_BATCH_SIZE)

# Set model

In [ ]:
model = get_model(cfg)

# Training and evaluation

In [ ]:
gpus = 1 if device == "cuda" else 0
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min")
trainer = pl.Trainer(max_epochs=cfg.SOLVER.MAX_EPOCHS, gpus=gpus, logger=tb_logger, callbacks=[checkpoint_callback])
trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)
trainer.test(test_dataloaders=test_loader)